In [1]:
import http.client, urllib, base64, requests
import cognitive_face as CF 
from cognitive_face import util
import time

In [2]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

/Users/Amiros/anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [3]:
#Emotion Analysis parameters
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognize'
_key = '183aea69820e4dfda0176db816fc1f72' #Here you have to paste your primary key
_maxNumRetries = 5

#Face detection parametrs
CF.Key.set('d78211e12bba41a991382ade5d68745b') #set the key

time: 3.23 ms


In [4]:
# Image path
urlImage= ''

# Emtional Anlaysis function
def processRequest( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result


headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None
params = None

time: 67.6 ms


In [5]:
# Face detect function
def detect(image, face_id=True, landmarks=False, faceRectangle= False, attributes='gender'):
    
    url = 'detect'
    headers, data, json = util.parse_image(image)
    params = {
        'returnFaceId': face_id and 'true',
        'returnFaceLandmarks': landmarks and 'false',
        'returnFaceAttributes': attributes,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)

time: 9.62 ms


In [6]:
#Get all the frames from S3
#start with 355
url_list = []
for i in range(355,5920,5):
    url = "https://s3-us-west-2.amazonaws.com/debateinemotion/debate1/frame%d.jpg" % i
    url_list.append(url)

time: 4.94 ms


In [12]:
# Face Detection & Emotioin Analysis 
emotions = {}
frame_index = {}
count = 355
#355

for single_image in url_list:

    json = { 'url': single_image } 
    emotions[count] = detect(single_image)
    processed = processRequest(json, data, headers, params)
    
    try:
        emotions[count].append(processed[0])  
    except:
        pass 

    if count < 5350:
        count += 5
    
    time.sleep(3)

time: 1h 41min 34s


In [20]:
sorted_emotions = sorted(emotions.items(), key=lambda s: s[0])

time: 2.23 ms


In [30]:
for key, value in sorted_emotions.items() :
    (key)

AttributeError: 'dict_keys' object has no attribute 'sort'

time: 8.37 ms


In [32]:
for key in sorted(emotions):
    print (key)


355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855
860
865
870
875
880
885
890
895
900
905
910
915
920
925
930
935
940
945
950
955
960
965
970
975
980
985
990
995
1000
1005
1010
1015
1020
1025
1030
1035
1040
1045
1050
1055
1060
1065
1070
1075
1080
1085
1090
1095
1100
1105
1110
1115
1120
1125
1130
1135
1140
1145
1150
1155
1160
1165
1170
1175
1180
1185
1190
1195
1200
1205
1210
1215
1220
1225
1230
1235
1240
1245
1250
1255
1260
1265
1270
1275
1280
1285
1290
1295
1300
1305
1310
1315
1320
1325
1330
1335
1340
1345
1350
1355
1360
1365
1370
1375
1380
1385
1390
1395
1400
1405
1410
1415
1420
1425
1430
1435
1440
1445
1450
1455
1460
1465
1470
1475
1480

In [14]:
# Save the result as Json
import json
with open('debate1_full.json', 'w') as fp:
    json.dump(emotions, fp, sort_keys=True)

time: 192 ms
